In [5]:
import sys                                                                              #THIS IS FOR GOOGLE COLAB USERS ONLY
from google.colab import files
sys.path.insert(0,'/content/drive/MyDrive/Product-Title-Classification')

In [6]:
import sys                                                                              #THIS IS FOR GOOGLE COLAB USERS ONLY
from google.colab import files
sys.path.insert(0,'/content/drive/MyDrive/Product-Title-Classification')
!cp /content/drive/MyDrive/Product-Title-Classification/Utilities.py /content           #THIS IS FOR GOOGLE COLAB USERS ONLY
!cp /content/drive/MyDrive/Product-Title-Classification/KNNImpute.py /content

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Local Files

In [8]:
import KNNImpute
import Utilities

Importing Local Files

In [ ]:
# from KNNImpute import *
# from Utilities import *

## Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import math

## Retrieving Training Data

In [ ]:
labels=["country", "sku_id", "title", "category_lvl1","category_lvl2","category_lvl3", "description", "price", "type"]

def getTrainingDataset():
    return pd.read_csv('/content/drive/MyDrive/Product-Title-Classification/data_train.csv', header=None, names=labels)  


raw_data = getTrainingDataset()
train_df,Y1,Y2,Y3= Utilities.Cleaning_Data_Utility(raw_data)                              
print(train_df.head())

# Imputing Null Values

In [ ]:
def NullStatistics(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

NullStatistics(train_df)

unique_label_c1, unique_label_c2, unique_label_c3 = KNNImpute.preserve_label(train_df) #It has all unique values lying in ctg1 , 2 , 3 column
KNNImpute.encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],train_df)        #Performed encoding for CTGLVL3 KNN
train_df_imputed = KNNImpute.impute(train_df)
train_df_imputed =  KNNImpute.clean_csv(train_df_imputed,train_df)
NullStatistics(train_df_imputed)

#Splitting Data Into Training And Testing

In [12]:
X = train_df_imputed['Title_desc']
Y1 = train_df_imputed['category_lvl1']
Y2 = train_df_imputed['category_lvl2']
Y3 = train_df_imputed['category_lvl3']

#Feature Extraction

In [13]:
feature_extraction = TfidfVectorizer(min_df = 1, stop_words = 'english' , lowercase = 'True')

X_train_features = feature_extraction.fit_transform(X)

In [14]:
#Splitting Data Into Training And Testing

X_train, X_test , Y_train_cat_1 , Y_test_cat_1 = train_test_split(X_train_features, Y1, test_size = 0.3, random_state=3 , stratify = Y1)

#Applying Random Forest Classifier

In [15]:
#Predicting Value of n_estimators

X_train.shape
number_of_trees = X_train.shape
estimated_trees = math.sqrt(number_of_trees[0])
estimated_trees = int(math.floor(estimated_trees))
print(estimated_trees)

159


In [16]:
#Training Model For Category 1

model_cat_1 = RandomForestClassifier(n_estimators = estimated_trees) 
model_cat_1.fit(X_train, Y_train_cat_1)

RandomForestClassifier(n_estimators=159)

In [15]:
#Predicting On Testing Data

Y_pred_cat_1 = model_cat_1.predict(X_test)
accuracy_on_testing_data_cat_1 = accuracy_score(Y_test_cat_1 , Y_pred_cat_1)
print(accuracy_on_testing_data_cat_1)

0.9032613688562242


In [16]:
#Display Confusion Matrix

print('Accuracy score with Random Forest Classifier is : ')
print(metrics.accuracy_score(Y_test_cat_1 , Y_pred_cat_1)*100)
mat_cat_1 = confusion_matrix(Y_test_cat_1, Y_pred_cat_1)
print(mat_cat_1)

Accuracy score with Random Forest Classifier is : 
90.32613688562242
[[ 500   10    5    3   33    1   33    0    1]
 [   5  747    1    2   25    1   50   46    1]
 [   0    2 1636   10   53    0    5    1   12]
 [   0    0   31 1051  110    7    4    5    4]
 [   5    7   50   37 1664    8   22   10   11]
 [   0    0    3   11  119  335    7    0    0]
 [   7    7    5    3   20    1 2115   23    2]
 [   6   21    1    4   56    3   56  585   21]
 [   0    0   21    3   30    0    8    4 1199]]


In [ ]:
# Printing Classification Report

print(classification_report(Y_test_cat_1, Y_pred_cat_1))

              precision    recall  f1-score   support

         0.0       0.95      0.86      0.90       586
         1.0       0.94      0.85      0.89       878
         2.0       0.93      0.95      0.94      1719
         3.0       0.93      0.87      0.90      1212
         4.0       0.79      0.91      0.85      1814
         5.0       0.96      0.72      0.82       475
         6.0       0.91      0.97      0.94      2183
         7.0       0.88      0.77      0.82       753
         8.0       0.96      0.95      0.96      1265

    accuracy                           0.90     10885
   macro avg       0.92      0.87      0.89     10885
weighted avg       0.91      0.90      0.90     10885



# CATEGORY LEVEL 2

In [17]:
#Splitting Data into Training And Testing

X_train, X_test , Y_train_cat_2 , Y_test_cat_2 = train_test_split(X_train_features, Y2 , test_size = 0.3, random_state=3 , stratify = Y2)

In [47]:
#Calculating N Estimators

number_of_trees = X_train.shape
estimated_trees = math.sqrt(number_of_trees[0])
estimated_trees = int(math.floor(estimated_trees))
print(estimated_trees)

159


In [18]:
#Training Model for Category 2

model_cat_2 = RandomForestClassifier(n_estimators = estimated_trees) 
model_cat_2.fit(X_train, Y_train_cat_2)

RandomForestClassifier(n_estimators=159)

In [33]:
#Predicting On Testing Data

Y_pred_cat_2 = model_cat_2.predict(X_test)
accuracy_on_testing_data_cat_2 = accuracy_score(Y_test_cat_2 , Y_pred_cat_2)
print(accuracy_on_testing_data_cat_2)

0.8220486908589802


In [ ]:
#Display Confusion Matrix

print('Accuracy score with Random Forest Classifier is : ')
print(metrics.accuracy_score(Y_test_cat_2, Y_pred_cat_2)*100)
mat_cat_2 = confusion_matrix(Y_test_cat_2, Y_pred_cat_2)
print(mat_cat_2)

# CATEGORY LEVEL 3

In [19]:
#Splitting Data Into Training and Testing

X_train, X_test , Y_train_cat_3 , Y_test_cat_3 = train_test_split(X_train_features, Y3 , test_size = 0.3, random_state=3 , stratify = Y3)

In [36]:
#Calculating N Estimators

number_of_trees = X_train.shape
estimated_trees = math.sqrt(number_of_trees[0])
estimated_trees = int(math.floor(estimated_trees))
print(estimated_trees)

159


In [20]:
#Training Model for Category 3

model_cat_3 = RandomForestClassifier(n_estimators = estimated_trees) 
model_cat_3.fit(X_train, Y_train_cat_3)

RandomForestClassifier(n_estimators=159)

In [ ]:
#Predicting On Testing Data

Y_pred_cat_3 = model_cat_3.predict(X_test)
accuracy_on_testing_data_cat_3 = accuracy_score(Y_test_cat_3 , Y_pred_cat_3)
print(accuracy_on_testing_data_cat_3)

In [ ]:
#Display Confusion Matrix

print('Accuracy score with Random Forest Classifier is : ')
print(metrics.accuracy_score(Y_test_cat_3 , Y_pred_cat_3)*100)
mat_cat_3 = confusion_matrix(Y_test_cat_3, Y_pred_cat_3)
print(mat_cat_3)

Accuracy score with Random Forest Classifier is : 
73.32108406063391
[[  5   0   0 ...   0   0   0]
 [  0 106   0 ...   0   0   1]
 [  0   0  13 ...   0   0   0]
 ...
 [  0   0   0 ...  16   1   0]
 [  0   0   0 ...   0   7   0]
 [  0   0   0 ...   0   0  19]]


# TESTING DATA CLEANING

In [ ]:
labels=["country", "sku_id", "title", "category_lvl1","category_lvl2","category_lvl3", "description", "price", "type"]

def getTestingDataset():
    return pd.read_csv('/content/drive/MyDrive/Product-Title-Classification/data_valid.csv', header=None, names=labels)  

raw_test_data = getTestingDataset()
test_df, Y1 , Y2 , Y3 = Utilities.Cleaning_Data_Utility(raw_test_data) #Utilities. hatadena yahan se 

# Imputing Null Values


In [ ]:
def NullStatistics(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

NullStatistics(test_df)

unique_label_c1, unique_label_c2, unique_label_c3 = KNNImpute.preserve_label(test_df) #It has all unique values lying in ctg1 , 2 , 3 column
KNNImpute.encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],test_df)        #Performed encoding for CTGLVL3 KNN
test_df_imputed = KNNImpute.impute(test_df)
test_df_imputed =  KNNImpute.clean_csv(test_df_imputed,test_df)
NullStatistics(test_df_imputed)

In [23]:
X = test_df_imputed['Title_desc']
Y1 = test_df_imputed['category_lvl1']
Y2 = test_df_imputed['category_lvl2']
Y3 = test_df_imputed['category_lvl3']

In [25]:
# Transforming Features

X_test_features = feature_extraction.transform(X)

# CATEGORY LEVEL 1

In [26]:
#Predicting On Testing Data

Y_Testing_Cat_1 = model_cat_1.predict(X_test_features)

In [28]:
#Checking Accuracy on Category 1 Model

accuracy_on_testing_data_cat_1 = accuracy_score(Y1 , Y_Testing_Cat_1)
print(accuracy_on_testing_data_cat_1)

0.8989694205102213


# CATEGORY LEVEL 2

In [29]:
#Prediction on Testing Data

Y_Testing_Cat_2 = model_cat_2.predict(X_test_features)

In [30]:
#Checking Accuracy on Category 2 Model

accuracy_on_testing_data_cat_2 = accuracy_score(Y2 , Y_Testing_Cat_2)
print(accuracy_on_testing_data_cat_2)

0.8144112181111675


# CATEGORY LEVEL 3


In [31]:
#Prediction on Testing Data

Y_Testing_Cat_3 = model_cat_3.predict(X_test_features)

In [32]:
#Checking Accuracy on Category 3 Model

accuracy_on_testing_data_cat_3 = accuracy_score(Y3 , Y_Testing_Cat_3)
print(accuracy_on_testing_data_cat_3)

0.7230106436898125
